In [17]:
!pip install geopandas

In [0]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import branca.colormap as cm
import datetime as dt

In [0]:
data = pd.read_csv("train.csv")

In [0]:
data["Dates"] = pd.to_datetime(data["Dates"])
data["Year"] = data["Dates"].dt.year
data["Month"] = data["Dates"].dt.month
data["Day"] = data["Dates"].dt.day

In [23]:
data.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Year,Month,Day
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,13
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,13


In [45]:
map_test = data.copy()
map_test = map_test[map_test["Year"] == 2015]
map_test = map_test[map_test["Month"] == 1]
map_test = map_test[map_test["Day"] == 5]
map_test = data.drop(columns=["Descript", "DayOfWeek", "Resolution", "Address"])
map_test.head()

,Dates,Category,PdDistrict,X,Y,Year,Month,Day
0,2015-05-13 23:53:00,WARRANTS,NORTHERN,-122.425892,37.774599,2015,5,13
1,2015-05-13 23:53:00,OTHER OFFENSES,NORTHERN,-122.425892,37.774599,2015,5,13
2,2015-05-13 23:33:00,OTHER OFFENSES,NORTHERN,-122.424363,37.800414,2015,5,13
3,2015-05-13 23:30:00,LARCENY/THEFT,NORTHERN,-122.426995,37.800873,2015,5,13
4,2015-05-13 23:30:00,LARCENY/THEFT,PARK,-122.438738,37.771541,2015,5,13


In [0]:
sfpd = gpd.read_file("current_police_districts.geojson")

In [42]:
sfpd=sfpd[["district","geometry"]]
sfpd.head()

,district,geometry
0,SOUTHERN,"MULTIPOLYGON (((-122.39186 37.79425, -122.3917..."
1,BAYVIEW,"MULTIPOLYGON (((-122.38098 37.76480, -122.3810..."
2,MISSION,"MULTIPOLYGON (((-122.40954 37.76932, -122.4086..."
3,NORTHERN,"MULTIPOLYGON (((-122.43379 37.80793, -122.4337..."
4,TENDERLOIN,"MULTIPOLYGON (((-122.40217 37.78626, -122.4171..."


In [0]:
center = [np.mean(map_test["X"]), np.mean(map_test["Y"]) ]

In [50]:
center

[-122.4226164550066, 37.77102029800673]

In [54]:
sfmap = folium.Map(location=center, zoom_start=12)
sfmap